# Question Answering

Dobbiamo definire in primis ciò che è il question answering che andremo a trattare.
Il question answering è ESTRATTIVO, ovvero che diamo un contesto e una domanda, tirando fuori dal contesto la risposta. E' diverso dal question answering generico (quello di chatGPT, insomma) che ti produce in output del testo a partire da ciò che ha appreso durante il training. Questo implica che il modello alla base non avrà un'architettura encoder-decoder

## Dataset

Il dataset che useremo è lo Standford Question Answering Dataset (SQuAD)

In [1]:
from datasets import load_dataset

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets = load_dataset("squad")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [3]:
example = raw_datasets["train"][1]
example

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [4]:
example["title"]

'University_of_Notre_Dame'

In [5]:
example["context"]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [6]:
example["answers"]

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [7]:
# in train set we don't have any chance for multiple answers or no answers
raw_datasets["train"].filter(lambda x : len(x["answers"]["text"]) !=1)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [8]:
raw_datasets["validation"].filter(lambda x : len(x["answers"]["text"]) !=1)[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


## Tokenizer

La tokenizzazione è un concetto molto interessante all'interno di questo genere di task. Come modello, useremo BERT. Siccome è QA estrattivo, non abbiamo bisogno di un'architettura encoder-decoder e ci facciamo bastare una encoder-only.

Il tokenizer di BERT abbiamo visto che è in grado di prendere due input (caso text entailment). Faremo una cosa simile anche qui, inserendo prima la domanda e poi il context dove è contenuta la risposta. Tuttavia, può accadere che il context sia eccessivamente lungo, così tanto da superare il limite della finestra di contesto del tokenizer di BERT.

Una soluzione valida è quella di spezzare il context in diverse window. Così facendo, passiamo al modello contemporaneamente più input (question+window) e la risposta cadrà in una delle finestre.

C'è un problema: cosa succede se una risposta cade a metà tra le finestre? Inizia a diventare un po' un caos, in quanto il modello avrà pezzi di risposta in input diversi e non sarà in grado di trovarla. Per questo motivo, risolviamo "sovrapponendo" le finestre. E lo faremo in modo che ogni finestra sia sovrapposta a quelle precedenti esattamente a metà. Questo crea ridondanza degli input ma ci assicura di trovare risposta.

In [9]:
from transformers import AutoTokenizer

In [10]:
model_checkpoint = "distilbert-base-cased"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
context = example["context"]
question = example["question"]

In [13]:
inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building \' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [14]:
inputs = tokenizer(
    question,
    context,
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)

In [15]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [16]:
# Qui è 0 perchè le finestre appartengono tutte allo stesso
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0]

In [17]:
inputs = tokenizer(
    raw_datasets["train"][:3]["question"],
    raw_datasets["train"][:3]["context"],
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)

In [18]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

In [19]:
inputs = tokenizer(
    question,
    context,
    truncation="only_second", # Effettua troncamento solo sul context (second input) e non sulla question
    max_length=100, # Lunghezza massima finestra 
    stride=50, # Quanto c'è di overlapping tra le finestre
    return_overflowing_tokens=True, # Qui diciamo che ci interessano i token della parte  overlappata
    return_offsets_mapping=True # Ci ritorna l'offset che esiste tra le windows e i tokens
)
for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building ' s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basilica is the Grotto

In [20]:
# Possiamo notare una cosa: questo output è una lista di liste di tuple
# Ad ogni indice tutto inzia sempre con la domanda, ma poi quando inizia il context il conteggio della posizione si riporta
# esattamente alla posizione contenuta nella frase originale, non alla posizione dell'input.
# P.s: i token speciali non hanno uno spazio, motivo per il quale si indicano con (0,0)
print(inputs["offset_mapping"])

[[(0, 0), (0, 4), (5, 7), (8, 10), (11, 16), (17, 19), (20, 23), (24, 29), (30, 34), (35, 39), (40, 48), (48, 49), (0, 0), (0, 13), (13, 15), (15, 16), (17, 20), (21, 27), (28, 31), (32, 33), (34, 42), (43, 52), (52, 53), (54, 56), (56, 58), (59, 62), (63, 67), (68, 76), (76, 77), (77, 78), (79, 83), (84, 88), (89, 91), (92, 93), (94, 100), (101, 107), (108, 110), (111, 114), (115, 121), (122, 126), (126, 127), (128, 139), (140, 142), (143, 148), (149, 151), (152, 155), (156, 160), (161, 169), (170, 173), (174, 180), (181, 183), (183, 184), (185, 187), (188, 189), (190, 196), (197, 203), (204, 206), (207, 213), (214, 218), (219, 223), (224, 226), (226, 229), (229, 232), (233, 237), (238, 241), (242, 248), (249, 250), (250, 251), (251, 254), (254, 256), (257, 259), (260, 262), (263, 264), (264, 265), (265, 268), (268, 269), (269, 270), (271, 275), (276, 278), (279, 282), (283, 287), (288, 296), (297, 299), (300, 303), (304, 312), (313, 315), (316, 319), (320, 326), (327, 332), (332, 333

### Allineamento dei target

Dobbiamo allineare i target, ora. Questo perchè la posizione dell'answer varia da window a window, adesso. Dobbiamo prendere la posizione assoluta dell'answer e sottrarre l'offset della window

In [21]:
import numpy as np

In [22]:
# Questa funzione (sequence_ids) prende in input un indice (indice della lista esterna) e restituisce gli id.
# Dove troviamo 0, c'è la question. Dove c'è 1, sta la window di contesto. Altrimenti sono token speciali.
# Se usassimo BERT, potremmo avere questi campi direttamente nel "token_type_id"
print(inputs.sequence_ids(0), len(inputs.sequence_ids(0)))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None] 100


In [23]:
# Quel è il problema? Che dividendo la finestra di contesto, la posizione della risposta parte a 181.
# Questo significa che la risposta è presente sia nella finestra 100-200 che in quella 150 2500.
# nel primo caso la lunghezza relativa è 81, nel secondo 31
answer = raw_datasets["train"][1]["answers"]
answer

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [24]:
# Il tipo è una lista
type(inputs.sequence_ids(0))

list

In [25]:
sequence_ids = inputs.sequence_ids(0)

In [26]:
# Prima occorrenza di 1
sequence_ids.index(1)

13

In [27]:
# Invertiamo i sequence ids e vediamo dove si trova l'1 ora:
# Siccome l'ultimo token è il SEP, avremo ovviamente che la stringa al contrario è il primo.
# Usiamo questa informazione per tagliare, dalla lunghezza massima, il valore di indice
sequence_ids[::-1].index(1), len(sequence_ids) - sequence_ids[::-1].index(1) -1

(1, 98)

In [28]:
# in alternativa...
np.argwhere(sequence_ids)[0], np.argwhere(sequence_ids)[-1]

(array([13], dtype=int64), array([98], dtype=int64))

In [29]:
# ora vogliamo usare questa funzione in combinazione con gli ids per trovare il context:
sequence_ids = inputs.sequence_ids(0)
ctx_start = sequence_ids.index(1) # -> Ci dice l'indice della prima occorrenza dell'1
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) -1

ctx_start, ctx_end

(13, 98)

In [30]:
# Ora arriviamo a trovare la posizione della risposta nelle finestre di contesto, a partire dall'indice della risposta nella
# lista. quindi mapperemo la risposta da "char index" a "token index" e poi da "token index (full context) a "window token idx".
# Se non è presente nella finestra, usremo (0,0) come coordinata del token [spazio nullo]
# Faremo questa cosa staticamente per solo la prima finestra. Poi inseriremo tutto in un loop
ans_start_char = answer["answer_start"][0] #188
ans_end_char = ans_start_char + len(answer["text"][0]) #188+ len

offset = inputs["offset_mapping"][0]
start_idx, end_idx = 0, 0
# RICORDA: Offset = tupla (x, y) dove il primo valore (x) è l'indice di partenza del token, y è l'indice di arrivo
# Se il primo valore di offset della prima parola è maggiore dell'indice di inizio della risposta 
# OPPURE se l'ultimo valore di offset dell'ultima parola è minore dell'indice di fine della risposta
if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
    print("(0, 0)") # Fai nulla
else:
    # Troviamo la posizione relativa della rispsota nella finestra di contesto.
    # Loopiamo negli offset, cerchiamo linearmente i token all'interno della tupla
    i = ctx_start # Partiamo da qui per evitare di trovare la posizione nella domanda.
    for start_end_char in offset[ctx_start:]:
        start, end = start_end_char
        if start == ans_start_char:
            start_idx = i
        if end == ans_end_char:
            end_idx = i
            break # il break solo qui
        i+=1

start_idx, end_idx

(53, 57)

In [31]:
# vediamo se è corretto:
input_ids = inputs["input_ids"][0]
input_ids[start_idx:end_idx+1] , tokenizer.decode(input_ids[start_idx:end_idx+1])

([170, 7335, 5921, 1104, 4028], 'a copper statue of Christ')

In [32]:
# Scriviamoci la funzione
def find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset):
    start_idx, end_idx = 0, 0
    #Se la risposta non è interamente contenuta nel context:
    if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
        pass
        # Ritorna 0, 0 come settato all'inizio
    else:
        i = ctx_start
        for start_end_char in offset[ctx_start:]:
            start, end = start_end_char
            if start == ans_start_char:
                start_idx = i
            if end == ans_end_char:
                end_idx = i
                break # il break solo qui
            i+=1
    
    return start_idx, end_idx

In [33]:
# Mettiamo tutto in un loop per tutte le finestre di contesto, usando questa funzione!

start_idxs, end_idxs = [], []
for i, offset in enumerate(inputs['offset_mapping']):
    sequence_ids = inputs.sequence_ids(i)
    context_ids = np.argwhere(sequence_ids)
    ctx_start, ctx_end = context_ids[0][0], int(context_ids[-1][0])

    start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)
    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

## Tokenizer

A differenza del solito, questa volta i tokenizer saranno due. Uno in fase di training, uno in fase di evaluating.
Key points:
1. Dobbiamo creare le context window
2. Siccome molti dati raggiungeranno la max length, il padding lo inseriamo nel tokenizer e sarà uguale per tutti
3. Dobbiamo allineare le answer alle context window

Il motivo dei due tokenizer è perchè facciamo operazioni diverse sugli output. Nel tokenizer di validazione, infatti, le metriche lavoreranno direttamente sulle stringhe, non sugli IDs

### Train Tokenizer

In [34]:
# In primo luogo, alcune domande sono mal formattate ed hanno degli spazi prima e dopo. Vediamo quali:
for q in raw_datasets["train"]["question"][:1000]:
    if q.strip() != q:
        print(q)

In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [35]:
 # Questi valori vengono da esperimenti di google:
max_lenght = 384
stride = 128

def tokenizer_fn_train(batch):

    questions = [q.strip() for q in batch["question"]]

    inputs = tokenizer(
        questions,
        batch["context"],
        max_length=max_lenght,
        stride=stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Siccome non avremo bisogno delle colonne dopo, le rimuoviamo per non ritrovarcele in uscita
    offset_mapping = inputs.pop("offset_mapping")
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    answers = batch["answers"]

    start_idxs, end_idxs = [], []

    for i, offset in enumerate(offset_mapping):
        # Qui noi stiamo semplicemente recuperando la risposta dal dataset originale (raw) perchè il tokenizer crea quello espanso
        sample_idx = orig_sample_idxs[i]
        answer = answers[sample_idx]
        
        ans_start_char = answer["answer_start"][0]
        ans_end_char = ans_start_char + len(answer["text"][0])
        
        sequence_ids = inputs.sequence_ids(i)
        context_ids = np.argwhere(sequence_ids)
        ctx_start, ctx_end = context_ids[0][0], int(context_ids[-1][0])

        start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)
        start_idxs.append(start_idx)
        end_idxs.append(end_idx)

    inputs["start_positions"] = start_idxs
    inputs["end_positions"] = end_idxs
    return inputs    

In [36]:
train_dataset = raw_datasets["train"].map(tokenizer_fn_train, batched=True, remove_columns=raw_datasets["train"].column_names)

In [37]:
len(raw_datasets["train"]), len(train_dataset)

(87599, 88729)

### Eval Tokenizer

Il tokenizer della validazione non necessita dei target. La risposta sarò comparata direttamente sulle stringhe

Una seconda differenza riguarda l'offset mapping: metteremo a "None" i token relativi alla domanda, scopriremo perchè dopo

In [38]:
# Terremo gli "id", gli identificatori univoci del campione nel dato. Serviranno per matchare correttamente in fase di eval
raw_datasets["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [39]:
def tokenizer_fn_validation(batch):

    questions = [q.strip() for q in batch["question"]]

    inputs = tokenizer(
        questions,
        batch["context"],
        max_length=max_lenght,
        stride=stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Siccome non avremo bisogno delle colonne dopo, le rimuoviamo per non ritrovarcele in uscita
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    sample_ids = [] # Qui ci saranno gli id alfanumerici

    for i in range(len(inputs["input_ids"])):
        sample_idx = orig_sample_idxs[i]
        sample_ids.append(batch["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Lascia l'offset invariato se è relativo al contesto, altrimenti diventa un None
        inputs["offset_mapping"][i] = [x if sequence_ids[j] == 1 else None for j, x  in enumerate(offset)]

    inputs["sample_id"] = sample_ids

    return inputs    

In [40]:
validation_dataset = raw_datasets["validation"].map(
    tokenizer_fn_validation, batched=True, remove_columns=raw_datasets["validation"].column_names
)

In [41]:
len(raw_datasets["validation"]), len(validation_dataset)

(10570, 10822)

## Metrics

A differenza di altre metriche, qui non andiamo a comparare la posizione dei token a livello quantitativo. La metrica che andremo ad usare è una metrica legata al dataset e fa il confronto tra la risposta ottenuta e la risposta reale.
Questo significa che dovremo convertire le risposte del modello in stringhe.

Altra cosa importante è cosa restituisce la metrica. Ci da due valori: il primo è un exact matching (#chars predetti bene / #total chars), l'altra è una metrica più smooth che ammette a risposte vicine a quelle sbagliate di avere un bonus.

La metrica si aspetta due input: una list di dizionari come predizioni e una lista di dizionari per le references. Queste due però sono diverse.

In [42]:
from evaluate import load

In [43]:
metric = load("squad")

In [44]:
predicted_answers = [
    {'id': '1', 'prediction_text': 'Albert Einstein'},
    {'id': '2', 'prediction_text': 'physicist'},
    {'id': '3', 'prediction_text': 'general relativity'}
]

true_answers = [
    {'id': '1', 'answers': {'text': ['Albert Einstein'], 'answer_start':[100]}},
    {'id': '2', 'answers': {'text': ['physicist'], 'answer_start':[100]}},
    {'id': '3', 'answers': {'text': ['special relativity'], 'answer_start':[100]}}
]
    
metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 66.66666666666667, 'f1': 83.33333333333333}

### Conversione Logit -> Stringhe

Attualmente stiamo creando la funzione per convertire i logit in stringhe. In particolare creiamo proprio la "compute_metrics". Per testarla, proviamo un modello già addestrato.

In [45]:
import torch
from transformers import AutoModelForQuestionAnswering

In [46]:
small_validation_dataset = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

In [47]:
# Facciamo sto switch perchè il tokenizer si riferisce ad una global variable chiamata "tokenizer"
old_tokenizer = tokenizer
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
small_validation_processed = small_validation_dataset.map(
    tokenizer_fn_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)
    
tokenizer = old_tokenizer

In [49]:
# Portiamo in torch
small_model_inputs = small_validation_processed.remove_columns(["sample_id", "offset_mapping"])
small_model_inputs.set_format("torch")

In [50]:
# Portiamo in GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
small_model_inputs_gpu = {
    k: small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}

In [51]:
# Scarichiamo il modello
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

In [52]:
print(small_model_inputs_gpu)

{'input_ids': tensor([[  101,  5979,  4279,  ...,     0,     0,     0],
        [  101,  5979,  4279,  ...,     0,     0,     0],
        [  101,  2777,  1225,  ...,     0,     0,     0],
        ...,
        [  101,  1327,  1160,  ...,     0,     0,     0],
        [  101,  2627,  3012,  ...,     0,     0,     0],
        [  101,  2627, 21188,  ...,     0,     0,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}


In [53]:
# Otteniamo gli output senza aggiornare il modello
# Il doppio asterisco converte il dizionario small_model_inputs_gpu negli argomenti del modello
# E' come se facesse trained_model(input_ids=x, attentiona_mask=y)
with torch.no_grad():
    outputs = trained_model(**small_model_inputs_gpu)

In [54]:
# L'oggetto è particolare
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -2.2607,  -5.1783,  -5.2709,  ...,  -9.5243,  -9.5183,  -9.5288],
        [ -2.5961,  -5.5482,  -5.5313,  ...,  -9.9598,  -9.9533,  -9.9860],
        [ -3.7127,  -7.1848,  -8.5388,  ..., -11.6557, -11.6571, -11.6505],
        ...,
        [ -2.0260,  -4.4167,  -4.4980,  ...,  -8.1479,  -8.1530,  -8.1760],
        [ -4.1553,  -5.8304,  -7.1643,  ..., -10.5255, -10.5251, -10.4890],
        [ -3.2000,  -5.8162,  -6.7249,  ...,  -9.4935,  -9.5038,  -9.4871]],
       device='cuda:0'), end_logits=tensor([[ -0.7353,  -4.9236,  -5.1048,  ...,  -8.8734,  -8.8916,  -8.8550],
        [ -1.3056,  -5.3870,  -5.4945,  ...,  -9.4895,  -9.5039,  -9.4958],
        [ -2.7649,  -7.2201,  -9.0916,  ..., -11.3106, -11.3414, -11.2702],
        ...,
        [ -0.0768,  -4.8210,  -4.4374,  ...,  -8.0483,  -8.0502,  -7.9903],
        [ -2.7347,  -5.3650,  -7.2549,  ..., -10.0498, -10.0661,  -9.9886],
        [ -1.0991,  -4.2569,  -6.1267,  ...,  -8

In [55]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [56]:
# Ora vediamo la questione dei samples.
# Gli id sono delle stringhe alfanumeriche. Alcune di queste si ripetono nel dataset
print(small_validation_processed["sample_id"][:5])
print(len(validation_dataset["sample_id"]))
print(len(set(validation_dataset["sample_id"])))

['56be4db0acb8001400a502ec', '56be4db0acb8001400a502ed', '56be4db0acb8001400a502ee', '56be4db0acb8001400a502ef', '56be4db0acb8001400a502f0']
10822
10570


In [57]:
# Creiamo un mapping "string": numero campioni
sample_id2idxs = {}
for i, id_ in enumerate(small_validation_processed["sample_id"]):
    if id_ not in sample_id2idxs:
        sample_id2idxs[id_] = [i]
    else:
        sample_id2idxs[id_].append(i)

In [58]:
start_logits.shape, end_logits.shape

((100, 384), (100, 384))

In [59]:
# Questo per ricordarci come fare un ordering discendente di qualcosa basato su indici. 
# Ci serve per trovare gli n indici dei valori più alti agilmente
(-start_logits[0]).argsort()
start_logits[0][(-start_logits[0]).argsort()]

array([10.694445  ,  9.803685  ,  4.459973  ,  4.400487  ,  2.9437785 ,
        2.7017367 ,  2.0126448 ,  1.5780739 ,  0.52237445,  0.02073721,
       -0.02802688, -0.04971648, -0.38573122, -0.6945363 , -0.7979508 ,
       -0.86780477, -0.87220925, -1.3516886 , -1.3703715 , -1.3878827 ,
       -1.5135094 , -1.7355472 , -1.8827027 , -1.8932863 , -1.9078972 ,
       -1.9304978 , -2.2607322 , -2.2983866 , -2.3069332 , -2.5027428 ,
       -2.510063  , -2.530842  , -2.5399983 , -2.6718144 , -2.732354  ,
       -2.7710216 , -2.7713673 , -2.9521358 , -3.0604653 , -3.1706066 ,
       -3.204542  , -3.569336  , -3.5798059 , -3.6668851 , -3.7250628 ,
       -3.7498565 , -3.7632205 , -3.996814  , -4.0113316 , -4.0688004 ,
       -4.0944853 , -4.195475  , -4.2383103 , -4.3323617 , -4.352419  ,
       -4.3879614 , -4.38861   , -4.396615  , -4.6790547 , -4.7030315 ,
       -4.7757587 , -4.7778134 , -4.788218  , -4.7882495 , -4.8221273 ,
       -4.872539  , -4.8849363 , -4.8981495 , -5.072096  , -5.10

In [60]:
# Altro reminder: il nostro tokenizer inserisce dei "None" ovunque, tranne nella context window
# La context window è formata da una lista di tuple
# ogni tupla contiene (start_char_idx, end_char_idx) di ogni token
print(small_validation_processed["offset_mapping"][0])

[None, None, None, None, None, None, None, None, None, None, None, None, None, [0, 5], [6, 10], [11, 13], [14, 17], [18, 20], [21, 29], [30, 38], [39, 43], [44, 46], [47, 56], [57, 60], [61, 69], [70, 72], [73, 76], [77, 85], [86, 94], [95, 101], [102, 103], [103, 106], [106, 107], [108, 111], [112, 115], [116, 120], [121, 127], [127, 128], [129, 132], [133, 141], [142, 150], [151, 161], [162, 163], [163, 166], [166, 167], [168, 176], [177, 183], [184, 191], [192, 200], [201, 204], [205, 213], [214, 222], [223, 233], [234, 235], [235, 238], [238, 239], [240, 248], [249, 257], [258, 266], [267, 269], [269, 270], [270, 272], [273, 275], [276, 280], [281, 286], [287, 292], [293, 298], [299, 303], [304, 309], [309, 310], [311, 314], [315, 319], [320, 323], [324, 330], [331, 333], [334, 342], [343, 344], [344, 345], [346, 350], [350, 351], [352, 354], [355, 359], [359, 360], [360, 361], [362, 369], [370, 372], [373, 376], [377, 380], [381, 390], [391, 394], [395, 399], [400, 402], [403, 408

In [61]:
# Qui effettuiamo la conversione output - text
n_largest = 20 # Numero massimo di logits da controllare
max_answer_length = 30
predicted_answers = []

# Loopiamo sul dataset originale per prenderci l'answer originale
for sample in small_validation_dataset:
    sample_id = sample['id']
    context = sample["context"]
    
    # Inizializziamo gli score e le answer
    best_score = float('-inf')
    best_answer = None
    
    # Loopiamo nel mapping input samples per prenderci gli indici del dataset espanso corrispondenti
    # Da qui calcoliamo la probabilità massima della risposta
    for idx in sample_id2idxs[sample_id]:
        start_logit = start_logits[idx]
        end_logit = end_logits[idx]
        offsets = small_validation_processed[idx]["offset_mapping"]
        
        start_indices = (-start_logit).argsort()
        end_indices = (-end_logit).argsort()
        
        for start_idx in start_indices[:n_largest]:
            for end_idx in end_indices[:n_largest]:
                
                # Caso 1: Siamo fuori dalla context window e skippiamo
                if offsets[start_idx] is None or offsets[end_idx] is None:
                    continue

                # Caso 2: start >= end
                if end_idx < start_idx:
                    continue

                # Caso 3: risposta troppo lunga
                if end_idx - start_idx + 1 > max_answer_length:
                    continue

                # Calcolo dello score logaritmico dei logit.
                # argmax(log((p_si, p_ej))) -> l_si + l_ej
                score = start_logit[start_idx] + end_logit[end_idx]
                if score > best_score:
                    best_score = score
                    # Ora troviamo la posizione relativa ai caratteri
                    start_char = offsets[start_idx][0]
                    end_char = offsets[end_idx][1]
                    best_answer = context[start_char:end_char]
    predicted_answers.append({"id":sample_id, "prediction_text": best_answer})
                    

In [62]:
small_validation_dataset["answers"][0]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [63]:
predicted_answers[0]

{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'}

In [64]:
true_answers = [{"id" : x["id"], "answers": x["answers"]} for x in small_validation_dataset]

In [65]:
metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

### Compute Metric Function

In [66]:
from tqdm.autonotebook import tqdm

In [75]:
def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):
    sample_id2idxs = {}
    for i, id_ in enumerate(processed_dataset["sample_id"]):
        if id_ not in sample_id2idxs:
            sample_id2idxs[id_] = [i]
        else:
            sample_id2idxs[id_].append(i)

    predicted_answers = []
    # tqdm mostra solo una progress bar
    for sample in tqdm(orig_dataset):
        sample_id = sample['id']
        context = sample["context"]

        best_score = float("-inf")
        best_answer = None
        
        for idx in sample_id2idxs[sample_id]:
            start_logit = start_logits[idx]
            end_logit = end_logits[idx]
            # ATTENZIONE!! Qui, conviene usare prima [idx] e poi ["offset_mapping"] o ci mette un'eternità!
            offsets = processed_dataset[idx]["offset_mapping"]
            
            start_indices = (-start_logit).argsort()
            end_indices = (-end_logit).argsort()
            
            for start_idx in start_indices[:n_largest]:
                for end_idx in end_indices[:n_largest]:
                    
                    # Caso 1: Siamo fuori dalla context window e skippiamo
                    if offsets[start_idx] is None or offsets[end_idx] is None:
                        continue
    
                    # Caso 2: start >= end
                    if end_idx < start_idx:
                        continue
    
                    # Caso 3: risposta troppo lunga
                    if end_idx - start_idx + 1 > max_answer_length:
                        continue
    
                    # Calcolo dello score logaritmico dei logit.
                    # argmax(log((p_si, p_ej))) -> l_si + l_ej
                    score = start_logit[start_idx] + end_logit[end_idx]
                    if score > best_score:
                        best_score = score
                        # Ora troviamo la posizione relativa ai caratteri
                        start_char = offsets[start_idx][0]
                        end_char = offsets[end_idx][1]
                        best_answer = context[start_char:end_char]
        
        predicted_answers.append({"id":sample_id, "prediction_text": best_answer})

    true_answers = [{"id" : x["id"], "answers": x["answers"]} for x in orig_dataset]

    return metric.compute(predictions=predicted_answers, references=true_answers)

In [76]:
compute_metrics(start_logits, end_logits, small_validation_processed, small_validation_dataset)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 847.49it/s]


{'exact_match': 83.0, 'f1': 88.25000000000004}

## Train and Evaluate Model

In [77]:
from transformers import AutoModelForQuestionAnswering, Trainer, TrainingArguments

In [78]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
args = TrainingArguments(
    "finetuned_squad",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

In [80]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset.shuffle(seed=42).select(range(1000)), #Non campionare se vuoi risultati buoni
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=375, training_loss=3.555673828125, metrics={'train_runtime': 82.2567, 'train_samples_per_second': 36.471, 'train_steps_per_second': 4.559, 'total_flos': 293969475072000.0, 'train_loss': 3.555673828125, 'epoch': 3.0})

In [81]:
trainer_output = trainer.predict(validation_dataset)

In [82]:
type(trainer_output)

transformers.trainer_utils.PredictionOutput

In [83]:
trainer_output

PredictionOutput(predictions=(array([[-0.66015625,  0.45581055, -0.31225586, ..., -5.3320312 ,
        -5.40625   , -5.3945312 ],
       [-0.66064453,  0.4711914 , -0.27026367, ..., -5.3320312 ,
        -5.40625   , -5.3945312 ],
       [-0.6826172 ,  0.26464844, -1.9990234 , ..., -5.390625  ,
        -5.359375  , -5.4101562 ],
       ...,
       [-0.7294922 , -0.484375  , -2.875     , ..., -5.0859375 ,
        -5.4570312 , -5.4804688 ],
       [-0.6977539 , -0.58935547, -1.0605469 , ..., -5.3632812 ,
        -5.3515625 , -5.3515625 ],
       [-0.71191406, -0.6845703 , -2.8613281 , ..., -5.0859375 ,
        -5.46875   , -5.484375  ]], dtype=float32), array([[-0.15002441, -1.6738281 , -1.3349609 , ..., -5.3515625 ,
        -5.3164062 , -5.234375  ],
       [-0.15161133, -1.6660156 , -1.3017578 , ..., -5.3515625 ,
        -5.3164062 , -5.2382812 ],
       [-0.16015625, -1.0322266 , -2.8046875 , ..., -5.3164062 ,
        -5.3203125 , -5.1953125 ],
       ...,
       [-0.15234375, -1.42480

In [84]:
predictions, label_ids, metrics = trainer_output

In [85]:
# Sappiamo che le predizioni sono start ed end logits.
predictions

(array([[-0.66015625,  0.45581055, -0.31225586, ..., -5.3320312 ,
         -5.40625   , -5.3945312 ],
        [-0.66064453,  0.4711914 , -0.27026367, ..., -5.3320312 ,
         -5.40625   , -5.3945312 ],
        [-0.6826172 ,  0.26464844, -1.9990234 , ..., -5.390625  ,
         -5.359375  , -5.4101562 ],
        ...,
        [-0.7294922 , -0.484375  , -2.875     , ..., -5.0859375 ,
         -5.4570312 , -5.4804688 ],
        [-0.6977539 , -0.58935547, -1.0605469 , ..., -5.3632812 ,
         -5.3515625 , -5.3515625 ],
        [-0.71191406, -0.6845703 , -2.8613281 , ..., -5.0859375 ,
         -5.46875   , -5.484375  ]], dtype=float32),
 array([[-0.15002441, -1.6738281 , -1.3349609 , ..., -5.3515625 ,
         -5.3164062 , -5.234375  ],
        [-0.15161133, -1.6660156 , -1.3017578 , ..., -5.3515625 ,
         -5.3164062 , -5.2382812 ],
        [-0.16015625, -1.0322266 , -2.8046875 , ..., -5.3164062 ,
         -5.3203125 , -5.1953125 ],
        ...,
        [-0.15234375, -1.4248047 , -4.3

In [86]:
start_logits, end_logits = predictions

In [87]:
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

100%|███████████████████████████████████████████████████████████████████████████| 10570/10570 [00:12<00:00, 837.73it/s]


{'exact_match': 13.907284768211921, 'f1': 23.692258097963602}

In [88]:
trainer.save_model("my_squad")

## Inference

In [89]:
from transformers import pipeline

In [90]:
qa = pipeline(
    "question-answering",
    model="my_squad",
    device=0
)

In [91]:
context = "Today I went to the store to purchase a carton of milk."
question = "What did I buy?"

qa(context=context, question=question)

{'score': 0.06326202303171158, 'start': 38, 'end': 46, 'answer': 'a carton'}